---

# Visualisation based on CSPP portifolio as a whole

## Visualisation 1: Number of (non-)green bonds in CSPP portifolio by time

### 1. Preparation and Import Data

**Finally `ggplot`!**
- Python has a module `plotnine` that supports `ggplot` kernel!    
    `%pip install plotnine`

**1.1 Load modules and dataset**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from plotnine import *
from mizani.breaks import date_breaks
from mizani.formatters import date_format

%matplotlib inline

In [ ]:
data_dir = '/Users/jingpuchen/Desktop/KU Leuven/Semester4/Modern Data Analytics/project_europa-main/data/final/'
file1 = '7_CSPPholdings_201706_2021.csv'
file2 = '7_info_match_round3_complete.csv'

- There are two dateframes, one is on company level, one is based on purchasement records.

In [ ]:
df = pd.read_csv(data_dir+file2, sep=',')
essential_variables = ['Name_1','OrgName_1','Name_2','OrgName_2','Primary Industry',
                       'Primary Bussiness','Primary Economic',
                       'Domiciled','Incorporated','TRBC code','green']
df_company = df[essential_variables]
df_purchase = pd.read_csv(data_dir+file1, sep=',')

### 2. Data wrangling

No wonder, `file_date` should be converted to datetime type

In [ ]:
df_purchase['file_date'] = pd.to_datetime(df_purchase['file_date'], format='%Y%m%d')

`count_data <- data %>% select(file_date, green) %>%
  group_by(file_date) %>% summarise(
    n_total = n(),
    n_green = sum(green ==1),
    n_nogreen = n_total - n_green
  )`

The following codes do the exact same job as above **R** commands, basically count the number of green purchasement and none green ones by each filing date. Even though filing date is not the same date when EU originally bought the bonds, it is the only information that we have.

In [ ]:
count_green_df = df_purchase.filter(['file_date', 'green']).groupby('file_date').agg(['count','mean'])
count_green_df.columns = count_green_df.columns.get_level_values(1)
count_green_df = count_green_df.assign(n_green=lambda x: x['count']*x['mean']).copy()
count_green_df['n_green'] = count_green_df['n_green'].astype(int)
count_green_df = count_green_df.assign(n_nogreen=lambda x: x['count']-x['n_green']).copy()

In [ ]:
count_green_df_small = count_green_df[['n_green','n_nogreen']]
count_green_df_small.reset_index(level=0, inplace=True)

`count_gather <- count_data %>% select(c(file_date, n_nogreen, n_green)) %>%
  gather(key = "variable", value = "value", -file_date)`

The following codes do the exact same job as above **R** commands, basically compress redundant columns into key-value pairs. This should be done because in the stage of visualising time series plot, we would like to have two distinct lines accoring to the company is green or not. 

In [ ]:
count_green_df_small_melt = pd.melt(count_green_df_small, id_vars="file_date")

`
theme_set(theme_minimal())
ggplot(count_gather, aes(x = file_date, y = value)) + 
  geom_area(aes(color = variable, fill = variable), 
            alpha = 0.3, position = 'stack', show.legend=TRUE) +
  scale_color_manual(values = c("forestgreen","orangered3")) +
  scale_fill_manual(values = c("forestgreen","orangered3")) +
  scale_x_date(limit=c(as.Date("2017-06-23"),as.Date("2021-04-16")),
               date_breaks = "3 month",date_labels = "%m/%Y") +
  scale_y_continuous(limits=c(400,1800),breaks=seq(400,1800,200)) +
  labs(x="",y="",title='CSPP_Holdings purchasement total and green') +
  theme(axis.text.x = element_text(angle=60, hjust=1),
        legend.title = element_text(size=10), #change legend title font size
        legend.text = element_text(size=10),  #change legend text font size
        legend.position="bottom", legend.box = "horizontal",
        legend.background = element_rect(fill="white",
                                         size=0.5, linetype="solid",
                                         colour ="black"))
`

`plotnine` enables us to code as if we were abusing ggplot system in Python. Nevertheless, all `.` should be replaced with `-`, while another important precaution is that variable name is surrounded by `'` when specifying them in python. 

### 3. PLOT

In [ ]:
theme_set(theme_minimal())
ggplot(count_green_df_small_melt, aes(x = 'file_date', y = 'value')) +\
  geom_area(aes(color = 'variable', fill = 'variable'), 
            alpha = 0.3, position = 'stack') +\
  scale_color_manual(values = ["forestgreen","orangered"]) +\
  scale_fill_manual(values = ["forestgreen","orangered"]) +\
  scale_x_datetime(breaks=date_breaks('3 months'),labels=date_format('%m/%Y')) +\
  labs(x="",y="",title='CSPP_Holdings purchasement total and green') +\
  theme(axis_text_x=element_text(angle=60, hjust=0.7),
        legend_title = element_text(size=10), #change legend title font size
        legend_text = element_text(size=10),  #change legend text font size
        legend_position="right", legend_box = "horizontal",
        legend_background = element_rect(fill="white",
                                         size=0.5, linetype="solid",
                                         colour ="black"))

---

## Visualisation 2: Ratio of green bonds over total in CSPP portifolio by time

In [ ]:
count_green_df_index = count_green_df.reset_index(level=0, inplace=False).copy()

### PLOT

In [ ]:
theme_set(theme_minimal())
ggplot(count_green_df_index, aes(x = 'file_date', y = 'mean')) +\
  geom_line(color='forestgreen',size=2) +\
  scale_y_continuous(limits=(0.45,0.50)) +\
  scale_x_datetime(breaks=date_breaks('3 months'),labels=date_format('%m/%Y')) +\
  labs(x="",y="",title='Proportion of Green Company Purchasement') +\
  theme(axis_text_x=element_text(angle=60, hjust=0.7))

---

# Visualisation based on new purchasement

## Data Cleaning

In [ ]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('7_CSPPholdings_201706_2021.csv')

This is a function to extract date information and convert it to a datetime object:

In [ ]:
def date(val):
    year = int(str(val)[0:4])
    month = int(str(val)[4:6])
    day = int(str(val)[6:8])
    date = datetime.datetime(year, month, day)
    return date
df['Date'] = df['file_date'].apply(date)

In order to filter the dataset to keep only the first record for each bond (that of the date when it was purchased), the dataset is grouped by ISIN code (unique for each bond) and the data is filter to keep only the minimum date.

In [ ]:
grouped = df[['ISIN_CODE', 'Date']]
grouped_df = grouped.groupby(by='ISIN_CODE').min().reset_index()
data = df[
    df[['ISIN_CODE','Date']].apply(tuple, axis = 1).isin(grouped_df[['ISIN_CODE', 'Date']].apply(tuple, axis = 1))
]

A column 'not_green' is created where 1 indicates the bond is related to a green economic sector and 0 otherwise.

In [ ]:
dataset = data[['green', 'Date']]
dataset['not_green'] = dataset.apply(lambda row: (row.green - 1)*(-1), axis=1)

The sum of green and non-green bonds is computed for each date. Moreover, the first date from the dataset is removed since it includes bonds from previous periods (cumulative figures).

In [ ]:
green_counts_df = dataset.groupby('Date').sum().reset_index()
green_counts_df = green_counts_df.drop(green_counts_df.index[0])
green_counts_df = green_counts_df.reset_index().drop('index', axis = 1)

## PLOTS

### 1. Plot of green and non-green bond purchases (figures by end of week)

In [ ]:
plt.plot('Date', 'green', data = green_counts_df)
plt.plot('Date', 'not_green', data = green_counts_df)
plt.legend()

Since the plot above is hard to read, the format of the date variable is transformed to only show year and month.

In [ ]:
green_counts_df['Date'] = green_counts_df['Date'].dt.date.apply(lambda x: x.strftime('%Y-%m'))

Now purchases figures are computed for each month within a year:

In [ ]:
green_counts_df2 = green_counts_df.groupby('Date').sum().reset_index()

### 2. Plot of green and non-green bond purchases (figures by month)

In [ ]:
plt.plot('Date', 'green', data = green_counts_df2)
plt.plot('Date', 'not_green', data = green_counts_df2)
plt.legend()

The plot above illustrates the evolution of bond purchases better. Additionally, individual plots for each year will be shown.

In [ ]:
green_counts_df3 = green_counts_df2.copy()
green_counts_df3['Year'] = green_counts_df3.apply(lambda row: int(row.Date[0:4]) , axis=1)

Splitting the data into multiple datasets by year:

In [ ]:
counts_2017 = green_counts_df3[green_counts_df3['Year'] == 2017]
counts_2018 = green_counts_df3[green_counts_df3['Year'] == 2018]
counts_2019 = green_counts_df3[green_counts_df3['Year'] == 2019]
counts_2020 = green_counts_df3[green_counts_df3['Year'] == 2020]
counts_2021 = green_counts_df3[green_counts_df3['Year'] == 2021]

### 3. Plot of green and non-green bond purchases in 2017 (monthly figures)

In [ ]:
plt.plot('Date', 'green', data = counts_2017)
plt.plot('Date', 'not_green', data = counts_2017)
plt.legend()
plt.title('Bond purchases in 2017')
plt.xticks(rotation=90)
plt.ylabel('Number of bonds acquired')
plt.xlabel('Date')

### 4. Plot of green and non-green bond purchases in 2018 (monthly figures)

In [ ]:
plt.plot('Date', 'green', data = counts_2018)
plt.plot('Date', 'not_green', data = counts_2018)
plt.legend()
plt.title('Bond purchases in 2018')
plt.xticks(rotation=90)
plt.ylabel('Number of bonds acquired')
plt.xlabel('Date')

### 5. Plot of green and non-green bond purchases in 2019 (monthly figures)

In [ ]:
plt.plot('Date', 'green', data = counts_2019)
plt.plot('Date', 'not_green', data = counts_2019)
plt.legend()
plt.title('Bond purchases in 2019')
plt.xticks(rotation=90)
plt.ylabel('Number of bonds acquired')
plt.xlabel('Date')

### 6. Plot of green and non-green bond purchases in 2020 (monthly figures)

In [ ]:
plt.plot('Date', 'green', data = counts_2020)
plt.plot('Date', 'not_green', data = counts_2020)
plt.legend()
plt.title('Bond purchases in 2020')
plt.xticks(rotation=90)
plt.ylabel('Number of bonds acquired')
plt.xlabel('Date')

#### Plot of green and non-green bond purchases in 2021 (monthly figures)

In [ ]:
plt.plot('Date', 'green', data = counts_2021)
plt.plot('Date', 'not_green', data = counts_2021)
plt.legend()
plt.title('Bond purchases in 2021')
plt.xticks(rotation=90)
plt.ylabel('Number of bonds acquired')
plt.xlabel('Date')

---

# Visualisation: Word Cloud

## **1 Load modules and dataset**

In [ ]:
# general management
import os, glob, chardet, json  
from itertools import compress
from wordcloud import WordCloud, ImageColorGenerator, STOPWORDS
from PIL import Image

# manage data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# nlp libraries
import re, nltk
nltk.download('stopwords') # for removing stopwords
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer # for stemming

# pre-processing data
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split


In [ ]:
path ='C:/Users/alawa/Desktop/Master Statistics/2nd year/Q2/Modern data analysis/project1'
full_path =  path + '/7_info_match_round3_complete.csv'    
df = pd.read_csv(full_path, sep=',')

outlook the variables of interest

In [ ]:
print(df['Primary Industry description'])

replacing NaN's with empty string

In [ ]:
df['Primary Industry description'] = df['Primary Industry description'].fillna(' ')

In [ ]:
print(df['Primary Industry description'])

## 2. Text cleaning

### 2.1 creating corpus

standardizing data

In [ ]:
corpus = []

for i in range(df.shape[0]):
    
    # preliminary
    IndDesc = re.sub('[^a-zA-Z]', ' ', df['Primary Industry description'][i]) # replace 'punctuation' with ' '
    IndDesc = IndDesc.lower()
    IndDesc = IndDesc.split()
    
    # remove stopwords and stemming process
    all_stopwords = stopwords.words('english')
    all_stopwords.remove('not') # do not remove 'not'
    
    # don't apply stemming to stopwords
    ps = PorterStemmer()
    IndDesc = [ ps.stem(word) for word in IndDesc if not word in set(all_stopwords) ] 
    
    # join all words again
    IndDesc = ' '.join(IndDesc)
    corpus.append(IndDesc)

Generating wordclouds

In [ ]:
# Generate a word cloud for green
green = list(compress(corpus, df['green']==1))
green_words = WordCloud(background_color="white", width=2000, height=1500).generate(str(green))

# Generate a word cloud for non-green
nongreen = list(compress(corpus, df['green']==0))
nongreen_words = WordCloud(background_color="white", width=2000, height=1500).generate(str(nongreen))

testing if they have the same length

## 3. PLOTS

In [ ]:
# Display the generated image:
plt.figure(figsize=(15,15))
plt.imshow(green_words, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
# Display the generated image:
plt.figure(figsize=(15,15))
plt.imshow(nongreen_words, interpolation="bilinear")
plt.axis("off")
plt.show()

---

# VISUALISATION DONE!